In [1]:
import tensorflow as tf
import numpy as np


In [2]:
def cnn_model_fn(features, labels, mode, params):
    x = features["x"]
    
    reshape = tf.reshape(x, [-1, 28, 28, 1])
    conv1 = tf.layers.conv2d(inputs=reshape, filters=32, kernel_size=[3, 3], padding="valid", activation=tf.nn.relu)
    pool2d = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    dropout_conv = tf.layers.dropout(pool2d, rate=params.dropout_rate, training=mode == tf.estimator.ModeKeys.TRAIN)
    conv1_flat = tf.layers.flatten(dropout_conv)
    dense1 = tf.layers.dense(inputs=conv1_flat, units=128, activation=tf.nn.relu)
    dropout_dense1 = tf.layers.dropout(dense1, rate=params.dropout_rate, training=mode == tf.estimator.ModeKeys.TRAIN)
    output = tf.layers.dense(inputs=dropout_dense1, units=10)

    predictions = {
      "classes": tf.argmax(input=output, axis=1),
      "probabilities": tf.nn.softmax(output, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=output)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
              labels=labels, predictions=predictions["classes"])}
        return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [3]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
mnist_classifier = tf.estimator.Estimator(
 model_fn=cnn_model_fn,
 model_dir="/tmp/mnist_convnet_model",
params={'dropout_rate': 0.5}
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f85e64597b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 2.3013692, step = 0
INFO:tensorflow:global_step/sec: 417.624
INFO:tensorflow:loss = 2.3077958, step = 100 (0.240 sec)
INFO:tensorflow:global_step/sec: 508.317
INFO:tensorflow:loss = 2.2490737, step = 200 (0.197 sec)
INFO:tensorflow:global_step/sec: 509.125
INFO:tensorflow:loss = 2.2459161, step = 300 (0.196 sec)
INFO:tensorflow:global_step/sec: 510.953
INFO:tensorflow:loss = 2.236266, step = 400 (0.196 sec)
INFO:tensorflow:global_step/sec: 509.902
INFO:tensorflow:loss = 2.1658683, step = 500 (0.196 sec)
INFO:tensorflow:global_step/sec: 513.241
INFO:tensorflow:loss = 2.18

In [6]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-13-09:36:24
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-13-09:36:24
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9294, global_step = 20000, loss = 0.24859141
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: /tmp/mnist_convnet_model/model.ckpt-20000
{'accuracy': 0.9294, 'loss': 0.24859141, 'global_step': 20000}
